# Detecção de Objetos Coloridos
Aluna: Arihé Redivo Ramos

Data: 16/06/2018

# Introdução

Vimos que no espaço de cor HSV, o canal H (*hue*, matiz) apresenta certa independência em relação à iluminação de um objeto. Essa característica pode ser explorada para a detecção de objetos. Teste o código seguinte.

In [12]:
# Adaptado de http://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html
import numpy as np
import cv2

frame = cv2.imread("mms.jpg") #obtida de http://blogs.mathworks.com/images/steve/2010/mms.jpg

# Convert BGR to HSV
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
# define range of blue color in HSV
blue=100 #importante: faixa da componente H no OpenCV: [0, 179]
thres=10
lower_blue = np.array([blue-thres,50,50])
upper_blue = np.array([blue+thres,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower_blue, upper_blue)
# Bitwise-AND mask and original image
res = cv2.bitwise_and(frame,frame, mask= mask)
cv2.imshow('frame',frame)
cv2.imshow('mask',mask)
cv2.imshow('res',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('mask.png', mask);
cv2.imwrite('res.png', res);

Imagem original:
![alt text](mms.jpg "Title")

Máscara originada pela detecção da cor azul:
![alt text](mask.png "Title")

Resultado do filtro:
![alt text](res.png "Title")

## Responda
1. Na função *cvtColor*, que outras conversões estão disponíveis?

    A função converte uma imagem de entrada de um espaço de cores para outro. No caso de uma transformação de para o espaço de cores RGB, a ordem dos canais deve ser especificada explicitamente (RGB ou BGR).
    Fonte:https://docs.opencv.org/3.1.0/d7/d1b/group__imgproc__misc.html#ga397ae87e1288a81d2363b61574eb8cab


2. O que faz a função *inRange*?

    Checa se os elementos de uma matriz está entre os elementos de outros dois arrays.
    
    A função verifica o intervalo da maneira seguinte, para cada elemento de uma matriz de entrada de canal único:
    
        dst(I) = lowerb(Io) ≤ src(Io) ≤ upperb(Io)
    
    A função verifica o intervalo da maneira seguinte, para matrizes de dois canais:
    
        dst(I) = lowerb(Io) ≤ src(Io) ≤ upperb(Io) ∧ lowerb(Ii) ≤ src(Ii) ≤ upperb(Ii) e assim por diante.
    
   Ou seja, dst (I) está configurado para 255 (todos os 1 bits) se src (I) estiver dentro da caixa especificada 1D, 2D, 3D, ... e 0 caso contrário.

    Fonte:https://docs.opencv.org/3.1.0/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981

Agora, vamos testar o mesmo procedimento usando um sinal de vídeo (webcam).

In [ ]:
import cv2
import numpy as np
color_to_find=120 #importante: faixa da componente H no OpenCV: [0, 179]
thres=10
cap = cv2.VideoCapture(0)
while(1):
    # Take each frame
    _, frame = cap.read()
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # define range of blue color in HSV

    lower_blue = np.array([color_to_find-thres,50,50])
    upper_blue = np.array([color_to_find+thres,255,255])
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

cv2.imwrite('frame2.jpg', frame );
cv2.imwrite('mask2.jpg', mask);
cv2.imwrite('res2.jpg', res);


Inclua, aqui, um quadro capturado e o resultado. Para isso, você deve gravar as imagens com *imwrite* e incorporar o link com o código mostrado abaixo. Ao enviar seu relatório, inclua as imagens no pacote zip.

Ultimo frame para a detecção azul:
![frame2](frame2.jpg "FRAME2")

Máscara do frame:
![mask2](mask2.jpg "MASK2")

Resultado do Filtro:
![res2](res2.jpg "RES2")

Você deve ter observado que há muito ruído na máscara. Uma forma de fazer uma "limpeza" é através de operações morfológicas. 

# Operações morfológicas

São operações baseadas no formato da imagem. Usam um *kernel* ou elemento estruturante que define a operação. Os operadores básicos são erosão e dilatação.

## Erosão

A ideia básica da erosão é retirar os pixels que estão nas bordas do objeto (objeto é o que está em branco, ou seja, com intensidade 1).

O elemento estruturante desliza sobre a imagem, como na convolução 2D. Um pixel na imagem resultado será 1 se todos os pixels no kernel forem 1, caso contrário, será erodido (intensidade 0).

Desse modo, os pixels próximos às bordas serão descartados. Quanto? Depende do tamanho do kernel. Essa operação é útil, por exemplo, para remover ruídos isolados de pequeno tamanho e separar dois objetos conectados.

Há vários exemplos em http://homepages.inf.ed.ac.uk/rbf/HIPR2/erode.htm

Observe o exemplo com um elemento estruturante 5x5.


In [18]:
#Exemplo de http://docs.opencv.org/3.2.0/d9/d61/tutorial_py_morphological_ops.html
import cv2
import numpy as np
img = cv2.imread('j.png',0)
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)
#inclua o código para visualização do resultado

## Dilatação

É o oposto da erosão. Aqui, um pixel resultado será 1 se pelo menos um dos pixels sob análise for 1. Desse modo, há um aumento do tamanho do objeto (aumento da região branca).

Dilatação é comumente aplicada após uma operação de erosão para redução de ruído. Também é útil para unir partes de um objeto.

In [19]:
dilation = cv2.dilate(img,kernel,iterations = 1)
#inclua o código para visualização do resultado

In [20]:
cv2.imwrite('erosion.png', erosion);
cv2.imwrite('dilation.png', dilation);

Imagem original:
![alt text](j.png "Title")
Eroção na Imagem:
![alt text](erosion.png "Title")
Dilatação na imagem:
![alt text](dilation.png "Title")

## Abertura e fechamento
São operações que usam erosão e dilatação.

## Atividade
Você vai fazer um contador automático de M&Ms.

Usando a imagem dos M&Ms (ou outra que você achar interessante), identifique os de cor verde. A primeira etapa é processar a imagem de modo que não exista conexão entre objetos próximos. Isso é necessário para a etapa seguinte, de contagem, a ser vista na próxima aula.

Inclua seu código e imagens de resultado.

In [58]:
# Adaptado de http://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html
import numpy as np
import cv2

frame = cv2.imread("fig_4.jpg") 

# Convert BGR to HSV
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# define range of blue color in HSV
green=44 #importante: faixa da componente H no OpenCV: [0, 179]
thres=10
lower_green = np.array([green-thres,50,50])
upper_green = np.array([green+thres,255,255])

# Threshold the HSV image to get only blue colkors
mask = cv2.inRange(hsv, lower_green, upper_green)

kernel = np.ones((3,3),np.uint8)

cv2.imshow('mask',mask)

mask = cv2.erode(mask,kernel,iterations = 2)
mask = cv2.dilate(mask,kernel,iterations = 2)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(frame,frame, mask= mask)
cv2.imshow('frame',frame)
cv2.imwrite('frame3.png', frame);
cv2.imshow('mask',mask)
cv2.imwrite('mask3.png', mask);
cv2.imshow('res',res)
cv2.imwrite('res3.png', res);

balinhas = 0

labelnum, labelimg, contours, GoCs = cv2.connectedComponentsWithStats(mask)
for label in xrange(1,labelnum):
    x,y,w,h,size = contours[label]
    if size >= 1000:
        frame2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255), 1)
    
    if size >= 4500:
        total = total + 1
    
    x,y = GoCs[label]
    frame2 = cv2.circle(frame2, (int(x),int(y)), 44, (255,255,255), 1) 

balinhas = balinhas + (labelnum-1)

cv2.imshow('frame2',frame2)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('frame4.png', frame2);

Imagem original:
![alt text](frame3.png "Title")

Máscara do frame para cor verde:
![alt text](mask3.png "Title")

Resultado do Filtro:
![alt text](res3.png "Title")

Detecção das balas verdes:
![alt text](frame4.png "Title")

Números de balas verdes indentificadas:

In [59]:
display(balinhas)

5